# Preprocessing SNSF Public Data

In [1]:
import os
import sqlite3
from sqlite3 import Error

import numpy as np
import pandas as pd

RAW_DATA_FOLDER = '../rawdata'
DATA_FOLDER = '../data/'

In [2]:
# file_names = os.listdir(RAW_DATA_FOLDER) # file_names[0], file_names[1]
project = pd.read_csv(os.path.join(RAW_DATA_FOLDER, 'P3_GrantExport.csv'), sep=';')
person = pd.read_csv(os.path.join(RAW_DATA_FOLDER, 'P3_PersonExport.csv'), sep=';')

## Assess

In [3]:
project.sample(3)

,Project Number,Project Number String,Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,Institution Country,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,All disciplines,Start Date,End Date,Approved Amount,Keywords
23967,53914,2000-053914,Sympas: Un Système de Modélisation Projective ...,NaN,Chabbi Drissi Houda,Project funding (Div. I-III),Project funding,Département d'Informatique Université de Fribourg,Switzerland,University of Fribourg - FR,20506,Information Technology,"Mathematics, Natural- and Engineering Sciences...",20506,01.01.1999,30.06.2001,85265.00,COMPUTER VISION; RECOGNITION; NUMERICAL INDERI...
64447,168083,PZ00P2_168083,Mineralogical processes and rate of serpentini...,Mineralogical processes and rate of serpentini...,Malvoisin Benjamin,Ambizione,Careers,Institut des sciences de la Terre Université d...,Switzerland,University of Lausanne - LA,20801,Geology,"Mathematics, Natural- and Engineering Sciences...",20801/20806,01.09.2016,31.01.2020,535564.00,serpentinization; mass transfer; fluid/rock in...
10080,25192,3100-025192,Studies on manipulated yeast minichromosomes: ...,NaN,Thoma Fritz,Project funding (Div. I-III),Project funding,Institut für Zellbiologie ETH Zürich,Switzerland,ETH Zurich - ETHZ,30104,Genetics,Biology and Medicine;Basic Biological Research,30104,01.10.1989,30.09.1993,415439.00,NaN


In [4]:
project.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74807 entries, 0 to 74806
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Project Number                74807 non-null  int64 
 1   Project Number String         74807 non-null  object
 2   Project Title                 74807 non-null  object
 3   Project Title English         31553 non-null  object
 4   Responsible Applicant         74807 non-null  object
 5   Funding Instrument            74807 non-null  object
 6   Funding Instrument Hierarchy  74767 non-null  object
 7   Institution                   69131 non-null  object
 8   Institution Country           69066 non-null  object
 9   University                    74802 non-null  object
 10  Discipline Number             74807 non-null  int64 
 11  Discipline Name               74807 non-null  object
 12  Discipline Name Hierarchy     74303 non-null  object
 13  All disciplines 

In [5]:
person.sample(3)

,Last Name,First Name,Gender,Institute Name,Institute Place,Person ID SNSF,OCRID,Projects as responsible Applicant,Projects as Applicant,Projects as Partner,Projects as Practice Partner,Projects as Employee,Projects as Contact Person
72618,Oetliker,Martina,female,Institut für chirurgische Technologien und Bio...,Bern,75789,NaN,NaN,47316,NaN,NaN,47316,NaN
37579,Grocprescu,Gabriela,female,Institut des sciences et ingénierie chimiques ...,Lausanne,122762,NaN,NaN,63876,NaN,NaN,NaN,NaN
77334,Pittet,Irène,female,NaN,NaN,726779,NaN,NaN,NaN,NaN,NaN,163859;190084,NaN


In [6]:
person.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112476 entries, 0 to 112475
Data columns (total 13 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   Last Name                          112476 non-null  object
 1   First Name                         112469 non-null  object
 2   Gender                             112476 non-null  object
 3   Institute Name                     54497 non-null   object
 4   Institute Place                    54393 non-null   object
 5   Person ID SNSF                     112476 non-null  int64 
 6   OCRID                              7328 non-null    object
 7   Projects as responsible Applicant  29056 non-null   object
 8   Projects as Applicant              18969 non-null   object
 9   Projects as Partner                5460 non-null    object
 10  Projects as Practice Partner       735 non-null     object
 11  Projects as Employee               82415 non-null   

### Tidiness

#### `project` (ie. `GrantExport`) table

- `Funding Instrument`, `Funding Instrument Hierarchy` are confusing
- `Discipline`, ... `Discipline Name Hierarchy` are confusing
- Details about `Institute` out of scope
- `University` contains both long and short names: details out of scope

#### `person` (ie. `PersonExport`) table

- [x]`Project as...` contain mixed variables and observations for grant and role
- Details about `Institute` out of scope

#### Quality

- [x] spaces in column names

##### `project` (ie. `GrantExport`) table

- [x] `Project Number` and `Project Number String` are redundant
- [x] `Project Number String` encodes division information?
- [x] `Responsible Applicant` not an uid
- [x] `Start Date` and `End Date` string, not date type
- [x] `Aproved Amount` not numeric
- [x] 2 observations without `start_date` or `end_date`
- `Project Title English` often redundant or null
- `Institution` free text? if yes, is it relevant? better named as department?
- `Keywords` not consistent (see keyword extraction from abstracts)

##### `person` (ie. `PersonExport`) table

- typo in col name for `ORCID`
- gender not categorical variable
- make sure `person_id_snsf` is unique

## Clean

In [7]:
# Fix column names
project.columns = project.columns.str.lower().str.replace(' ', '_')
person.columns = person.columns.str.lower().str.replace(' ', '_')

assert all(project.columns.str.contains(' ') == False), 'Space in project column names'
assert all(person.columns.str.contains(' ') == False), 'Space in person column names'
assert all(project.columns == project.columns.str.lower()), 'Uppercase in project column names'
assert all(person.columns == person.columns.str.lower()), 'Uppercase in person column names'

In [8]:
# https://github.com/zambujo/p3data/issues/4
role_types = np.array(person.columns[person.columns.str.contains('projects_as_')])
hr = (person
        .melt('person_id_snsf', 
              var_name='role', 
              value_name='project_number', 
              value_vars=role_types).dropna())
hr = (hr
        .assign(project_number=hr
        .project_number.str.split(';'))
        .explode('project_number')
        .reset_index(drop=True))
hr['role'] = hr.role.str[12:] # remove 'projects_as_'
hr['project_number'] = pd.to_numeric(hr['project_number'], errors='coerce')
hr=hr.dropna()
hr['project_number'] = hr['project_number'].astype(int)
hr.drop_duplicates(inplace=True)

# hr.role.value_counts()
assert hr.project_number.dtype.name == 'int64', 'project number column not an integer'

In [9]:
# hr table
hr.reset_index(inplace=True)
hr = hr.rename(columns = {'index':'id',
                          'project_number':'project_id', 
                          'person_id_snsf':'person_id'})
hr = hr[['id', 'project_id', 'person_id', 'role']]

assert all(hr.id.duplicated() == False), 'id contains duplicates'

In [10]:
# person table
person = person.rename(columns = {'person_id_snsf':'id'})
person = person[['id', 'gender', 'first_name', 'last_name']]
person = person.drop_duplicates()

assert all(person.id.duplicated() == False), 'id contains duplicates'

In [11]:
# https://github.com/zambujo/p3data/issues/1
# remove ending project_number >> remove ending separator(s)
project['string_code'] = (
    project.project_number_string.
        str.replace('([0-9]+)$', '').
        str.replace('[^a-zA-Z0-9]+$', ''))
project = project.drop('project_number_string', 1)

In [12]:
# string_code vs funding_instrument
# grants.groupby(["string_code", "funding_instrument"]).size().reset_index().rename(columns={0: "count"}).sort_values(by='count', ascending=False)

In [13]:
# https://github.com/zambujo/p3data/issues/6
project['approved_amount'] = pd.to_numeric(project['approved_amount'], errors='coerce')
project['start_date'] = pd.to_datetime(project.start_date)
project['end_date'] = pd.to_datetime(project.end_date)

assert project.approved_amount.dtype.name == 'float64', 'approved amount column not a float'
assert project.start_date.dtype.name == 'datetime64[ns]', 'start_date column not datetime type'
assert project.end_date.dtype.name == 'datetime64[ns]', 'end_date column not datetime type'

In [14]:
# https://github.com/zambujo/p3data/issues/13
project=project.dropna(subset=['start_date'])
project=project.dropna(subset=['end_date']) # to be on safe side

assert all(pd.isna(project.start_date) == False), 'NA values in project.start_date'
assert all(pd.isna(project.end_date) == False), 'NA values in project.end_date'

In [15]:
# project table
project[['domain','topic']] = project.discipline_name_hierarchy.str.split(';', expand=True)
project[['programme', 'programme_details', 'programme_specifics']] = project.funding_instrument_hierarchy.str.split(';', expand=True)

project = project.rename(columns = {'project_number':'id'})
project = project[['id', 'programme', 'funding_instrument', 'domain', 'topic', 'discipline_name', 'start_date', 'end_date', 'approved_amount']]

assert all(project.id.duplicated() == False), 'id contains duplicates'

In [18]:
# create a database connection and a cursor object to a SQLite database
conn = None
try:
    conn = sqlite3.connect('../data/p3.db')
    c = conn.cursor()
except Error as e:
    print(e)

In [19]:
sql_drop_project_table = 'DROP TABLE IF EXISTS project;'
sql_drop_person_table =  'DROP TABLE IF EXISTS person;'
sql_drop_hr_table =  'DROP TABLE IF EXISTS hr;'

sql_create_project_table = '''
CREATE TABLE IF NOT EXISTS project (
  id integer PRIMARY KEY,
  programme text,
  funding_instrument text,
  domain text,
  topic text,
  discipline_name text,
  start_date datetime,
  end_date datetime,
  approved_amount float
);'''

sql_create_person_table = '''
CREATE TABLE IF NOT EXISTS person (
  id integer PRIMARY KEY,
  gender text,
  first_name text,
  last_name text
);'''

sql_create_hr_table = '''
CREATE TABLE IF NOT EXISTS hr (
  id integer PRIMARY KEY,
  project_id integer,
  person_id integer,
  role text,
  FOREIGN KEY (project_id) REFERENCES project (id),
  FOREIGN KEY (person_id) REFERENCES person (id)
);'''

def manipulate_table(conn, sql_command):
    try:
        c = conn.cursor()
        c.execute(sql_command)
        # conn.commit()
    except Error as e:
        print(e)

if conn is not None:
    manipulate_table(conn, sql_drop_project_table)
    manipulate_table(conn, sql_drop_person_table)
    manipulate_table(conn, sql_drop_hr_table)
    manipulate_table(conn, sql_create_project_table)
    manipulate_table(conn, sql_create_person_table)
    manipulate_table(conn, sql_create_hr_table)
else:
    print("Problem with connection to database.")

In [20]:
project.to_sql('project', con=conn, if_exists='append', index=False)
person.to_sql('person', con=conn, if_exists='append', index=False)
hr.to_sql('hr', con=conn, if_exists='append', index=False)

In [ ]:
if conn is not None:
    conn.close()
else:
    print("Problem with connection to database.")